In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
MAX_MODEL_RUNTIME_MINS = 10
MAX_MODEL_RUNTIME_SECS = MAX_MODEL_RUNTIME_MINS * 60

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
import datatable as dt

train = dt.fread('/kaggle/input/tabular-playground-series-nov-2021/train.csv', max_nrows=500000)
test = dt.fread('/kaggle/input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
target = train['target'].to_numpy().ravel()

In [ ]:
h2o.init()

h2o_train = h2o.H2OFrame(train.to_pandas())
h2o_test = h2o.H2OFrame(test.to_pandas())

h2o_train['target'] = h2o.H2OFrame(target).asfactor()

In [ ]:
features = [x for x in h2o_train.columns if x != 'target']

model_h2oaml = H2OAutoML(stopping_metric='auc', max_runtime_secs=MAX_MODEL_RUNTIME_SECS)
model_h2oaml.train(x=features, y='target', training_frame=h2o_train)

In [ ]:
model_h2oaml.leaderboard

In [ ]:
preds_h2oaml = model_h2oaml.leader.predict(h2o_test).as_data_frame()['True']

In [ ]:
sample_sub = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

In [ ]:
sample_sub.head()

In [ ]:
sample_sub['target']=preds_h2oaml

In [ ]:
sample_sub.to_csv('submission.csv', index=False)